# Diabetes Prediction

## Loading Dataset

In [222]:
import pandas as pd

In [223]:
data = pd.read_csv('diabetes_prediction_dataset.csv')
data.head(5)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [224]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


## Exploratory Data Analysis

In [228]:
import seaborn as sns
import matplotlib.pyplot as plt

#### What is the Distribution of Diabetes Cases in this Data

## Data Preprocessing

In [225]:
import numpy as np

In [226]:
data.loc[data["smoking_history"] == "No Info", "smoking_history"] = np.nan
data.loc[data['smoking_history'] == 'current', 'smoking_history'] = 1
data.loc[data['smoking_history'] == 'never', 'smoking_history'] = 0
data.loc[data['smoking_history'] == 'ever', 'smoking_history'] = 0
data.loc[data['smoking_history'] == 'not current', 'smoking_history'] = 0
data.loc[data['smoking_history'] == 'former', 'smoking_history'] = 2


data.loc[data['gender'] == 'Male', 'gender'] = 1
data.loc[data['gender'] == 'Female', 'gender'] = 0
data.loc[data['gender'] == 'Other', 'gender'] = 2

In [227]:
data = data.dropna()

## Selecting the Best Machine Learning Algorithm

## Finding the Best Features

## Creating the Model

## Evaluating the Model

## Saving the Model

## Conclusion